In [ ]:
#!pip install praw
#!pip install db-sqlite3
#!pip install datetime
#!pip install pytz
#!pip install pandas
#!pip install schedule

In [ ]:
from datetime import date, datetime, timedelta
import praw
import prawcore
import re
import pandas as pd
import sqlite3
import time
import shutil
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import math

In [ ]:
reddit = praw.Reddit('bot1')
reddit.validate_on_submit = True

In [ ]:
# Generate database
# Needed tables:
# - Table with all previous posts. Name: posts. Columns: post_id (string), date (date)
# - Table with gems of each user. Name: gems. Columns: username (string), gems (int), date (date)
# - Table with current stocks and no. of posts at that moment of each user. Name: stocks. Columns: username (string), subreddit (string), amount (int), posts (int)
# - Backup table with all trades of users, and the no. of posts at these moments. Name: trades. Columns: username (string), subreddit (string), amount (int), posts (int), post_id (string), date (date)
# - Table with loans of each user. Name: loans. Columns: username (string), amount (int), date (date)
# - Backup table with all loan changes. Name: loans_backup. Columns: username (string), amount (int), type (string), date (date)
# - Table with all self-posted comments.
# - Table with all subreddit rates.

def create_database():
    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    
    # Create tables
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS posts (
            post_id TEXT PRIMARY KEY,
            date DATE
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS gems (
            username TEXT,
            gems INTEGER,
            date DATE,
            PRIMARY KEY (username, date)
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS stocks (
            username TEXT,
            subreddit TEXT,
            amount INTEGER,
            value FLOAT,
            PRIMARY KEY (username, subreddit)
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS trades (
            username TEXT,
            subreddit TEXT,
            amount INTEGER,
            value FLOAT,
            date DATE,
            type TEXT,
            PRIMARY KEY (username, subreddit, date, type)
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS loans (
            username TEXT PRIMARY KEY,
            amount INTEGER
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS loans_backup (
            username TEXT,
            amount INTEGER,
            type TEXT,
            date DATE,
            PRIMARY KEY (username, date, type)
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS comments (
            comment_id TEXT,
            date DATE,
            PRIMARY KEY (comment_id, date)
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS posts_per_subreddit (
            subreddit TEXT,
            date DATE,
            posts INT,
            PRIMARY KEY (subreddit, date)
        )
    ''')

    conn.commit()
    conn.close()
    
create_database()

In [ ]:
def isfloat(num):
    try:
        float(num)
        return True
    except ValueError:
        return False

In [ ]:
def add_row(df, username, message):
    new_row = pd.DataFrame([[username, message]], columns=df.columns)
    return pd.concat([df, new_row], ignore_index=True)

In [ ]:
def get_today():
    return date.today().isoformat()

In [ ]:
def get_latest_post():
    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()

    cursor.execute("""
        SELECT post_id, date
        FROM posts 
        ORDER BY date DESC 
        LIMIT 1
    """)
    latest_post = cursor.fetchone()
    post_id, post_date = latest_post
    conn.close()
    return (post_id, post_date)

In [ ]:
def subreddit_exists(subreddit_name):
    try:
        subreddit = reddit.subreddit(subreddit_name)
        # Access a public attribute to confirm accessibility
        subreddit.title  # This will raise an exception if the subreddit is banned or private
        return True
    except prawcore.exceptions.NotFound:
        return False  # Subreddit doesn't exist
    except prawcore.exceptions.Forbidden:
        return False  # Subreddit exists but is private
    except praw.exceptions.RedditAPIException:
        return False  # Subreddit is banned or restricted

In [ ]:
def extract_commands(text):
    pattern = re.compile(r'\\\[\s*(sell|buy)\s+(\d+)\s+r/(\w+)\s*\\\]|\\\[\s*(loan|pay)\s+(\d+)\s*\\\]|\\\[\s*(exit)\s*\\\]|\\\[(.*?)\\\]')
    
    commands = []
    for match in pattern.finditer(text.lower()):
        if match.group(1):  # sell or buy
            commands.append({
                'command': match.group(1),
                'amount': int(match.group(2)),
                'subreddit': match.group(3),
                'unrecognized': None
            })
        elif match.group(4):  # loan or pay
            commands.append({
                'command': match.group(4),
                'amount': int(match.group(5)),
                'subreddit': None,
                'unrecognized': None
            })
        elif match.group(6):  # exit
            commands.append({
                'command': match.group(6),
                'amount': None,
                'subreddit': None,
                'unrecognized': None
            })
        else:  # Unrecognized command
            commands.append({
                'command': None,
                'amount': None,
                'subreddit': None,
                'unrecognized': match.group(7)
            })
    
    if len(commands) == 0:
        pattern = re.compile(r'\[\s*(sell|buy)\s+(\d+)\s+r/(\w+)\s*\]|\[\s*(loan|pay)\s+(\d+)\s*\]|\[\s*(exit)\s*\]|\[(.*?)\]')
        for match in pattern.finditer(text.lower()):
            if match.group(1):  # sell or buy
                commands.append({
                    'command': match.group(1),
                    'amount': int(match.group(2)),
                    'subreddit': match.group(3),
                    'unrecognized': None
                })
            elif match.group(4):  # loan or pay
                commands.append({
                    'command': match.group(4),
                    'amount': int(match.group(5)),
                    'subreddit': None,
                    'unrecognized': None
                })
            elif match.group(6):  # exit
                commands.append({
                    'command': match.group(6),
                    'amount': None,
                    'subreddit': None,
                    'unrecognized': None
                })
            else:  # Unrecognized command
                commands.append({
                    'command': None,
                    'amount': None,
                    'subreddit': None,
                    'unrecognized': match.group(7)
                })
            
    return pd.DataFrame(commands)

In [ ]:
def user_is_player(username):
    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    cursor.execute("SELECT 1 FROM gems WHERE username = ? LIMIT 1", (username,))
    result = cursor.fetchone()
    conn.close()
    return result is not None

In [ ]:
def current_gems(username):
    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    cursor.execute("""
        SELECT gems FROM gems
        WHERE username = ?
        ORDER BY date DESC
        LIMIT 1
    """, (username,))
    current_gems = cursor.fetchone()[0]
    conn.close()
    return current_gems

In [ ]:
def add_gems(username, amount):
    gems = current_gems(username)
        
    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    cursor.execute("""
        SELECT date FROM gems
        WHERE username = ?
        ORDER BY date DESC
        LIMIT 1
    """, (username,))
    conn.commit()
    last_date = cursor.fetchone()[0]
    today = get_today()

    if today == last_date:     
        cursor.execute("UPDATE gems SET gems = ? WHERE username = ? AND date = ?", (gems + amount, username, today))
    else:
        cursor.execute("INSERT INTO gems (username, gems, date) VALUES (?, ?, ?)", (username, gems + amount, today))
    conn.commit()
    conn.close()


In [ ]:
def has_stocks(username, subreddit):
    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    cursor.execute("""
        SELECT 1 FROM stocks
        WHERE username = ?
        AND subreddit = ?
    """, (username,subreddit,))
    has_stocks = cursor.fetchone() is not None
    conn.close()
    
    return has_stocks

In [ ]:
def has_loan(username):
    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    cursor.execute("""
        SELECT 1 FROM loans
        WHERE username = ?
    """, (username,))
    has_loan = cursor.fetchone() is not None
    conn.close()
    
    return has_loan

In [ ]:
def to_unix_timestamp(date_string):
    # Format: 'YYYY-MM-DD HH:MM' (e.g., '2025-02-01 10:00')
    dt_object = datetime.strptime(date_string, '%Y-%m-%d %H:%M')
    return int(dt_object.timestamp())

In [ ]:
def get_posts_before_date(subreddit, date, username = 'test'):
    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    cursor.execute("""
        SELECT 1 FROM posts_per_subreddit
        WHERE LOWER(subreddit) = LOWER(?) AND date = ?
    """, (subreddit,date,))
    has_been_found = cursor.fetchone() is not None

    if has_been_found:
        cursor.execute("""
            SELECT posts FROM posts_per_subreddit
            WHERE LOWER(subreddit) = LOWER(?) AND date = ?
        """, (subreddit,date,))
        n_posts = cursor.fetchone()[0]
        conn.commit()
        conn.close()
        if username == "test":
            return n_posts
    elif username != 'test':
        raise Exception(f'I cannot find the post count of this subreddit ({subreddit}) and this user ({username}) on this date ({date}) yet!')
        
    end_date = datetime.strptime(date, "%Y-%m-%d") 

    end_datetime = datetime(end_date.year, end_date.month, end_date.day, 5, 0)
    start_datetime = end_datetime - timedelta(hours=24)

    start_timestamp = int(start_datetime.timestamp())
    end_timestamp = int(end_datetime.timestamp())

    # Count the number of posts in the time range
    post_count = 0
    
    if has_been_found:
        user = reddit.redditor(username)
        for submission in user.submissions.new():
            if submission.subreddit == subreddit and start_timestamp <= submission.created_utc < end_timestamp:
                post_count += 1
        return n_posts - post_count
    else:
        subreddit = reddit.subreddit(subreddit)

        # Loop through submissions in the subreddit
        for submission in subreddit.new(limit=1000):  # Use .new() to iterate through posts
            if start_timestamp <= submission.created_utc < end_timestamp:
                post_count += 1
            if submission.created_utc < start_timestamp:  # Stop early if past range
                break
        return post_count

# Call the function
#subreddit_name = 'Spookiz' # Replace with the desired subreddit name
#print(get_posts_before_date(subreddit_name, '2025-03-1'))
#print(get_posts_before_date(subreddit_name, '2025-02-28'))
#print(get_posts_before_date(subreddit_name, '2025-02-27'))
#print(get_posts_before_date(subreddit_name, '2025-02-26'))   

In [ ]:
def allowed_subreddits():
    words = ['dailygames','notinteresting', 'learnpython', 'mildlyinfuriating', '196', '3Blue1Brown', 'AmIOverreacting', 'AmITheAsshole', 'Angryupvote', 'Animal', 'animation', 'antimeme', 'anythingbutmetric', 'AskOuija', 'assholedesign', 'BeAmazed', 'birdification', 'birthofasub', 'blursedimages', 'brandnewsentence', 'capybara', 'CensoringIsHard', 'chemistrymemes', 'clevercomebacks', 'confidentlyincorrect', 'copypasta', 'countablepixels', 'Damnthatsinteresting', 'dataisbeautiful', 'DnD', 'dndmemes', 'ExplainTheJoke', 'facepalm', 'Fantasy', 'foundsatan', 'foundthemobileuser', 'FreeCompliments', 'gameofthrones', 'geocaching', 'girlsarentreal', 'GuysBeingDudes', 'iamverysmart', 'ididnthaveeggs', 'ihadastroke', 'im14andthisisdeep', 'Inroverts', 'interesting', 'interestingasfuck', 'LeftTheBurnerOn', 'LetGirlsHaveFun', 'lfg', 'lgbt', 'lies', 'linguisticshumor', 'LinkedInLunatics', 'lostredditors', 'MadeMeSmile', 'mapporncirclejerk', 'MathJokes', 'mathmemes', 'mattcolville', 'meirl', 'meme', 'memes', 'mildlyinteresting', 'MurderedByWords', 'nature', 'Nicegirls', 'NoahGetTheBoat', 'NonPoliticalTwitter', 'oddlyspecific', 'ofcoursethatsasub', 'offmychest', 'onejob', 'penpals', 'PeterExplainsTheJoke', 'pettyrevenge', 'physicsmemes', 'politics', 'PrematureTruncation', 'rareinsults', 'RoastMe', 'rpg', 'sciencememes', 'screenshotsarehard', 'softwaregore', 'sssdfg', 'SUBREDDITNAME', 'technicallythetruth', 'teenagersbutbetter', 'thatHappened', 'theydidthemath', 'Tinder', 'trolleyproblem', 'TwoSentenceHorror', 'vexillologycirclejerk', 'circlejerk', 'WeirdEggs', 'Whatcouldgowrong', 'whatisthisthing', 'woosh', 'wordle', 'AnarchyChess', 'shittydarksouls', 'KitchenConfidential', 'CountOnceADay', 'countwithchickenlady', 'SquaredCircle', 'chess']
    return sorted(words, key=str.lower)

In [ ]:
def is_allowed_subreddit(subreddit):
    return subreddit.lower() in [word.lower() for word in allowed_subreddits()]

# Not allowed: AITAH, teenagers

In [ ]:
def get_posts_per_subreddit(date):
    print("Get posts per subreddit")
    for i, subreddit in enumerate(allowed_subreddits()):
        print(f"Checking subreddit {i+1} out of {len(allowed_subreddits())}")
        n_posts = get_posts_before_date(subreddit, date, 'test')
        conn = sqlite3.connect("reddit_game.db")
        cursor = conn.cursor()
        cursor.execute("INSERT OR IGNORE INTO posts_per_subreddit (subreddit, date, posts) VALUES (?, ?, ?)", (subreddit, date, n_posts))
        conn.commit()
        conn.close()

In [ ]:
def add_player(username):
    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO gems (username, gems, date) VALUES (?, 1000, ?)", (username, get_today()))
    conn.commit()
    conn.close()
    return f"A new player: {username}, has joined. Welcome! You received 1000 gems."

In [ ]:
def buy(username, amount, subreddit, date):
    if not isfloat(amount):
        return f"{username} tried to buy {amount} stocks from r/{subreddit}, but this is not a whole number. The purchase has been cancelled."    
    amount = int(amount)
    
    gems = current_gems(username)
    number_of_posts = get_posts_before_date(subreddit, date, username)
    
    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    cursor.execute("""
        SELECT 1 FROM trades
        WHERE username = ?
        AND subreddit = ?
        AND date = ?
        AND type = ?
    """, (username,subreddit,date,"purchase",))
    has_already_bought_today = cursor.fetchone() is not None
        
    if has_already_bought_today:
        return f"{username} tried to buy stocks from r/{subreddit}, but has already done so below the same post. This is not possible, so the purchase has been cancelled."    
    if amount > gems:
        return f"{username} tried to buy {amount} stocks from r/{subreddit}, but only had {gems} gems. The purchase has been cancelled."
    if not is_allowed_subreddit(subreddit):
        return f"{username} tried to buy stocks from r/{subreddit}, but this subreddit is not in the list of allowed subreddits. The purchase has been cancelled."
#    if not subreddit_exists(subreddit):
#        return f"{username} tried to buy stocks from r/{subreddit}, but this subreddit has not been found (or it is for example restricted or banned). The purchase has been cancelled."
    if has_stocks(username, subreddit):
        return f"{username} tried to buy stocks from r/{subreddit}, but already has stocks from this subreddit. The purchase has been cancelled."
    if number_of_posts == 0:
        return f"{username} tried to buy stocks from r/{subreddit}, but there were 0 posts on this subreddit. This makes it impossible to determine the stock value. The purchase has been cancelled."
    
    add_gems(username,-1*amount)
    
    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO trades (username, subreddit, amount, value, date, type) VALUES (?, ?, ?, ?, ?, ?)", (username, subreddit, amount, 1/number_of_posts, date, "purchase"))
    conn.commit()
    cursor.execute("INSERT INTO stocks (username, subreddit, amount, value) VALUES (?, ?, ?, ?)", (username, subreddit, amount, 1/number_of_posts))
    conn.commit()
    conn.close()    
    
    return f"{username} bought {amount} stocks from r/{subreddit}. There have been {number_of_posts} posts (not posted by {username}), so that means each post is worth {1/number_of_posts:.5f} gems per stock."

In [ ]:
def sell(username, amount, subreddit, date):
    if not has_stocks(username, subreddit):
        return f"{username} tried to sell stocks from r/{subreddit}, but does not own any stocks from this subreddit. The sale has been cancelled."

    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    cursor.execute("""
        SELECT amount, value FROM stocks
        WHERE username = ?
        AND subreddit = ?
    """, (username,subreddit,))
    number_of_stocks, value = cursor.fetchone()
    conn.close()
    
    if amount == "all":
        amount = number_of_stocks
    if not isfloat(amount):
        return f"{username} tried to sell {amount} stocks from r/{subreddit}, but this is not a whole number. The sale has been cancelled."    
    amount = int(amount)
    if amount > number_of_stocks: # TODO: CHANGE SO IT JUST SETS IT TO THE MAXIMUM.
        return f"{username} tried to sell {amount} stocks from r/{subreddit}, but does only own {number_of_stocks} stocks. The sale has been cancelled."

    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    cursor.execute("""
        SELECT 1 FROM trades
        WHERE username = ?
        AND subreddit = ?
        AND date = ?
        AND type = ?
    """, (username,subreddit,date,"sale",))
    has_already_sold_today = cursor.fetchone() is not None
        
    if has_already_sold_today:
        return f"{username} tried to sell stocks from r/{subreddit}, but has already done so below the same post. This is not possible, so the sale has been cancelled."
    
    number_of_posts = get_posts_before_date(subreddit, date, username)
    gems = round(amount*number_of_posts*value)

    add_gems(username, gems)
    
    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    if number_of_posts == 0:
        value = 0
    else:
        value = 1/number_of_posts
    cursor.execute("INSERT INTO trades (username, subreddit, amount, value, date, type) VALUES (?, ?, ?, ?, ?, ?)", (username, subreddit, -1*amount, value, date, "sale"))
    conn.commit()
    if amount == number_of_stocks:   
        cursor.execute("DELETE FROM stocks WHERE username = ? AND subreddit = ?", (username, subreddit))
    else:
        cursor.execute("UPDATE stocks SET amount = ? WHERE username = ? AND subreddit = ?", (number_of_stocks - amount, username, subreddit))                
    conn.commit()
    conn.close()
    
    return f"{username} sold {amount} stocks from r/{subreddit}. There have been {number_of_posts} posts (not posted by {username}). Each post was worth {value:.5f} gems per stock. This means that this sale gave {username} {gems} gems."

In [ ]:
def loan(username, amount, date):
    if not isfloat(amount):
        return f"{username} tried to take a loan of {amount} gems, but this is not a whole number. The loan has not been granted."    
    amount = int(amount)
    
    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    cursor.execute("""
        SELECT 1 FROM loans_backup
        WHERE username = ?
        AND date = ?
        AND NOT type = ?
    """, (username,date,'interest',))
    has_already_worked_on_loan_today = cursor.fetchone() is not None
        
    if has_already_worked_on_loan_today:
        return f"{username} tried to get a loan, but they already got a loan/bought off a loan today. This is not possible on the same day, so no loan has been granted."
    
    add_gems(username, amount)

    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO loans_backup (username, amount, type, date) VALUES (?, ?, ?, ?)", (username, amount, 'loan', date))
    conn.commit()
    if has_loan(username):
        cursor.execute("""
            SELECT amount FROM loans
            WHERE username = ?
        """, (username,))
        current_loan = cursor.fetchone()[0]
        cursor.execute("UPDATE loans SET amount = ? WHERE username = ?", (current_loan + amount, username))                        
    else:
        cursor.execute("INSERT INTO loans (username, amount) VALUES (?, ?)", (username, amount))
    conn.commit()
    conn.close()
    
    return f"{username} took a loan of {amount} gems. They will have to pay an interest of {round(amount*0.05)} gems each day."

In [ ]:
def pay(username, amount, date):
    
    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    cursor.execute("""
        SELECT 1 FROM loans_backup
        WHERE username = ?
        AND date = ?
        AND NOT type = ?
    """, (username,date,'interest',))
    has_already_worked_on_loan_today = cursor.fetchone() is not None
        
    if has_already_worked_on_loan_today:
        return f"{username} tried to pay off a loan, but they already got a loan/bought off a loan today. This is not possible on the same day, so the payment has not been granted."
    
    gems = current_gems(username)
    
    if not has_loan(username):
        return f"{username} tried to pay back part of their loan, but they don't have a loan. The payback has been cancelled."
    
    cursor.execute("""
        SELECT amount FROM loans
        WHERE username = ?
    """, (username,))
    current_loan = cursor.fetchone()[0]

    if amount == "all":
        amount = current_loan
    if not isfloat(amount):
        return f"{username} tried to pay off {amount} gems from their loan, but this is not a whole number. This payment has not been granted."
    amount = int(amount)

    if amount > current_loan: # TODO: Change such that amount is put down.
        return f"{username} tried to pay back {amount} gems of their loan, but only {current_loan} gems of the loan were left. The payback has been cancelled."
    if amount > gems:
        return f"{username} tried to pay back {amount} gems of their loan, but only had {gems} gems. The payback has been cancelled."
    
    add_gems(username, amount*-1)

    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO loans_backup (username, amount, type, date) VALUES (?, ?, ?, ?)", (username, amount, 'payment', date))
    conn.commit()
    if amount == current_loan:
        cursor.execute("DELETE FROM loans WHERE username = ?", (username,))
    else:
        cursor.execute("UPDATE loans SET amount = ? WHERE username = ?", (current_loan - amount, username))                        
    conn.commit()
    conn.close()
    
    return f"{username} paid off {amount} gems of their loan. Now {current_loan - amount} gems are left in their loan. They will have to pay an interest of {round((current_loan-amount)*0.05)} gems each day."

In [ ]:
def exit(username):    
    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()    
    cursor.execute("DELETE FROM gems WHERE username = ?", (username,))
    cursor.execute("DELETE FROM stocks WHERE username = ?", (username,))
    cursor.execute("DELETE FROM trades WHERE username = ?", (username,))
    cursor.execute("DELETE FROM loans WHERE username = ?", (username,))
    cursor.execute("DELETE FROM loans_backup WHERE username = ?", (username,))
    conn.commit()
    conn.close()
    return f"{username} decided to exit the game. Their information has been deleted. Sorry to see you go. You're always welcome to join and start over again!"

In [ ]:
def unknown_command(username, command):
    return f"{username} gave me the following command: '{command}'. I do not know what to do, so no action has been taken."

In [ ]:
def execute_commands(username, commands):
    if username in ['B0tRank', 'WhyNotCollegeBoard']:
        return
    df = pd.DataFrame(columns=["username", "message"])
    if len(commands) == 0:
        return
    if not user_is_player(username):
        result = add_player(username)
        df = add_row(df,username,result)
    for index, row in commands.iterrows():
        print(f"Currently working on command {index+1} out of {len(commands)}")
        if row['command'] == 'buy':
            _, latest_post_date = get_latest_post()
            result = buy(username, row['amount'], row['subreddit'], latest_post_date)
            df = add_row(df,username,result)
        elif row['command'] == 'sell':
            result = sell(username, row['amount'], row['subreddit'], get_today())
            df = add_row(df,username,result)
        elif row['command'] == 'loan':
            result = loan(username, row['amount'], get_today())
            df = add_row(df,username,result)
        elif row['command'] == 'pay':
            result = pay(username, row['amount'], get_today())
            df = add_row(df,username,result)
        elif row['command'] == 'exit':
            result = exit(username)
            df = add_row(df,username,result)            
        elif row['command'] is None:
            result = unknown_command(username, row['unrecognized'])
            df = add_row(df,username,result)
    return df

In [ ]:
def format_messages(df):
    grouped = df.groupby('username')['message'].apply('\n\n'.join).reset_index()
    formatted_strings = grouped.apply(lambda row: f"u/{row['username']}\n\n{row['message']}", axis=1)
    return '\n\n---\n'.join(formatted_strings)

In [ ]:
#execute_commands("Aartvb",extract_commands("\[\]\[test\] hoi test \[buy 200 r/learnpython\]"))
#execute_commands("test123",extract_commands("hoi test \[buy 200 r/learnpython\]\[exit \]"))

In [ ]:
def increase_counter(i):
    i[0] += 1

In [ ]:
def pay_interest(execution_date):
    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    query = "SELECT username, amount FROM loans"
    df = pd.read_sql_query(query, conn)
    conn.close()    
    
    messages = pd.DataFrame(columns=["username", "message"])
    
    for index, row in df.iterrows():
        username = row['username']
        amount = row['amount']
        interest = round(amount*0.05)
        gems = current_gems(username)
        if gems >= interest:
            add_gems(username, interest*-1)
            messages = add_row(messages,username,f"{username} has paid {interest} gems as interest on their loan.")
        else:
            add_gems(username, gems*-1)
            loan_increase = interest - gems
            conn = sqlite3.connect("reddit_game.db")
            cursor = conn.cursor()
            cursor.execute("INSERT INTO loans_backup (username, amount, type, date) VALUES (?, ?, ?, ?)", (username, loan_increase, 'interest', execution_date))
            conn.commit()
            cursor.execute("UPDATE loans SET amount = ? WHERE username = ?", (amount + loan_increase, username))                        
            conn.commit()
            conn.close()
            messages = add_row(messages,username,f"{username} had to pay {interest} gems as interest on their loan. They only had {gems} gems. The rest has been added to their loan. Their loan is now {amount + loan_increase} gems, so they have to pay {round((amount + loan_increase)*0.05)} gems interest per day.")
    return messages

In [ ]:
def get_virtual_worth(username, date):
    worth = current_gems(username)
    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    query = "SELECT subreddit, amount, value FROM stocks WHERE username = ?"
    df = pd.read_sql_query(query, conn, params=(username,))
    conn.close()    
   
    print(f'getting virtual worth of {username}.')
    for index, row in df.iterrows():
        print(f"stock {index+1} out of {len(df)}.")
        subreddit = row['subreddit']
        amount = row['amount']    
        value = row['value']
    
        number_of_posts = get_posts_before_date(subreddit, date, username)
        worth += round(amount*number_of_posts*value)
    print("\n")
    return worth

In [ ]:
def get_current_rate(username, subreddit, amount, value):
    number_of_posts = get_posts_before_date(subreddit, get_today(), username)
    rate = round(amount*number_of_posts*value) - amount
    if rate > 0:
        rate = "+" + str(rate)
    else:
        rate = str(rate)
    return rate

In [ ]:
def create_gem_table():
    print("Creating gem table.")

    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    query = "SELECT username, gems, date FROM gems"
    df = pd.read_sql_query(query, conn)

    df['date'] = pd.to_datetime(df['date'])

    latest_df = df.sort_values(['username', 'date']).groupby('username').last().reset_index()
    latest_df = latest_df.drop('date',axis=1)

    conn.close()

    latest_df = latest_df.sort_values(['gems', 'username'], ascending = False)
    
    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    query = "SELECT username, amount FROM loans"
    df = pd.read_sql_query(query, conn)
    conn.close()

    n_columns = 2
    if len(df) > 0:
        latest_df["gems after interest"] = "-"
        n_columns = 3
        for index, row in df.iterrows():
            interest = round(row['amount']*0.05)
            gems = current_gems(row['username'])
            latest_df.loc[latest_df["username"] == row['username'], "gems after interest"] = round(gems-interest)
    
    # Create table image
    fig, ax = plt.subplots(figsize=(5, 2))  # Adjust size as needed
    fig.patch.set_facecolor('white')  # Ensure full white background
    ax.set_facecolor('white')  # Set axis background to white
    ax.set_title("Gems", fontsize=14, fontweight="bold", pad=15)  # **Title**
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=latest_df.values, colLabels=latest_df.columns, cellLoc='center', loc='center')
    
    # **Format the Table**
    table.auto_set_font_size(False)
    table.set_fontsize(8)
    
    # Make the first row bold
    for j in range(n_columns):  # columns
        table[0, j].set_text_props(fontweight="bold")

    # Save image
    plt.savefig("gems.png", dpi=300, bbox_inches="tight")
    
    plt.clf()
    plt.close('all')
#create_gem_table()

In [ ]:
def create_stock_table():
    print("Creating stock table.")

    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    query = "SELECT username, subreddit, amount, value FROM stocks"
    df = pd.read_sql_query(query, conn)
    conn.close()

    df = df.sort_values(['username', 'subreddit'])

    j = [0]
    df['current rate'] = [[print(f"row {j[0]+1} out of {len(df)}."), increase_counter(j), get_current_rate(row['username'], row['subreddit'], row['amount'], row['value'])][2]
           for i, row in df.iterrows()]

    df['value'] = df['value'].round(5)
    df = df.rename(columns = {'value':'gems/post/stock'})
    
    # Create table image
    fig, ax = plt.subplots(figsize=(9, 25))  # Adjust size as needed
    fig.patch.set_facecolor('white')  # Ensure full white background
    ax.set_facecolor('white')  # Set axis background to white
    ax.set_title("Stocks", fontsize=14, fontweight="bold", pad=15)  # **Title**
    ax.axis('tight')
    ax.axis('off')    
    table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')
    
    # **Format the Table**
    table.auto_set_font_size(False)
    table.set_fontsize(8)
    
    # Make the first row bold
    for j in range(5):  # 5 columns
        table[0, j].set_text_props(fontweight="bold")

    # Save image
    plt.savefig("stocks.png", dpi=300, bbox_inches="tight")
    
    plt.clf()
    plt.close('all')
#create_stock_table()

In [ ]:
def create_loan_table():
    print("Creating loan table.")

    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    query = "SELECT username, amount FROM loans"
    df = pd.read_sql_query(query, conn)
    conn.close()
    
    if len(df) == 0:
        return
    
    # Create table image
    fig, ax = plt.subplots(figsize=(5, 2))  # Adjust size as needed
    fig.patch.set_facecolor('white')  # Ensure full white background
    ax.set_facecolor('white')  # Set axis background to white
    ax.set_title("Loans", fontsize=14, fontweight="bold", pad=15)  # **Title**
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')
    
    # **Format the Table**
    table.auto_set_font_size(False)
    table.set_fontsize(8)
    
    # Make the first row bold
    for j in range(2):  # 2 columns
        table[0, j].set_text_props(fontweight="bold")

    # Save image
    plt.savefig("loans.png", dpi=300, bbox_inches="tight")
    
    plt.clf()
    plt.close('all')
#create_loan_table()

In [ ]:
def create_virtual_worth_table():
    print("Creating virtual worth table.")
    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    query = "SELECT DISTINCT username FROM gems"
    df = pd.read_sql_query(query, conn)

    df['virtual worth'] = df['username'].apply(lambda user: get_virtual_worth(user, get_today()))
    df = df.sort_values(['virtual worth', 'username'], ascending = False)

    # Create table image
    fig, ax = plt.subplots(figsize=(5, 2))  # Adjust size as needed
    fig.patch.set_facecolor('white')  # Ensure full white background
    ax.set_facecolor('white')  # Set axis background to white
    ax.set_title("Virtual worth (gems + current stock value)", fontsize=14, fontweight="bold", pad=15)  # **Title**
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')
    
    # **Format the Table**
    table.auto_set_font_size(False)
    table.set_fontsize(8)
    
    # Make the first row bold
    for j in range(2):  # 2 columns
        table[0, j].set_text_props(fontweight="bold")

    # Save image
    plt.savefig("virtual worth.png", dpi=300, bbox_inches="tight")
    
    plt.clf()
    plt.close('all')
#create_virtual_worth_table()

In [ ]:
def create_allowed_subreddit_image():
    print("Creating allowed subreddit image.")
    words = allowed_subreddits()
    columns = 3
    
    # Determine layout
    num_words = len(words)
    rows = math.ceil(num_words / columns)  # Calculate needed rows

    # Image size
    col_width = 450  # Width per column
    row_height = 40  # Height per row
    width = columns * col_width
    height = rows * row_height + 100  # Extra padding

    # Create image
    image = Image.new("RGB", (width, height), "white")
    draw = ImageDraw.Draw(image)

    # Load font
    try:
        title_font = ImageFont.truetype("arial.ttf", 50)  # Larger font for title
        font = ImageFont.truetype("arial.ttf", 30)  # Ensure font is available
    except IOError:
        title_font = ImageFont.load_default()
        font = ImageFont.load_default()

    # Draw title
    title = "Allowed Subreddits (and latest post number)"
    title_width, title_height = draw.textsize(title, font=title_font)
    draw.text(((width - title_width) // 2, 20), title, fill="black", font=title_font)

    # Draw words
    for index, word in enumerate(words):
        n_posts_today = get_posts_before_date(word, get_today(), 'test')
        n_posts_yesterday = get_posts_before_date(word, get_latest_post()[1], 'test')
        color = "black"
        if n_posts_today > n_posts_yesterday:
            color = "green"
        elif n_posts_today < n_posts_yesterday:
            color = "red"

        word = 'r/' + word + ": " + str(n_posts_today)
        col = index % columns
        row = index // columns
        x = col * col_width + 20  # Adjust X position
        y = row * row_height + 90  # Adjust Y position
        draw.text((x, y), word, fill=color, font=font)

    # Save image
    image.save("allowed subreddits.png")
#create_allowed_subreddit_image()

In [ ]:
def run_bot():
    shutil.copy2('reddit_game.db', f"reddit_game {datetime.now().strftime('%Y-%m-%d %H.%M.%S')}.db")
        
    post_id, post_date = get_latest_post()
    
    get_posts_per_subreddit(post_date)
    get_posts_per_subreddit(get_today())

    submission = reddit.submission(id=post_id)
    submission.comments.replace_more(limit=None)  # Load all nested comments

    df = pd.DataFrame(columns=["username", "message"])
    df = pd.concat([df, pay_interest(get_today())], ignore_index=True)
        
    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    query = "SELECT comment_id, date FROM comments"
    comments_to_ignore = pd.read_sql_query(query, conn)
    conn.close()    
    
    for comment in submission.comments.list():
        ignore_comment = False
        for index, row in comments_to_ignore.iterrows():
            if comment.id == row['comment_id'] and post_date == row['date']:
                ignore_comment = True
                break
        if ignore_comment:
            continue

        print("Working on comment:\n" + comment.body)

        df = pd.concat([df, execute_commands(comment.author.name,extract_commands(comment.body))], ignore_index=True)
        print("\n")

    df.sort_values(by=['username'])
    
    create_gem_table()
    create_stock_table()
    create_loan_table()
    create_virtual_worth_table()
    create_allowed_subreddit_image()

    change_log = format_messages(df)
    
    print('\n\n\n\n\n\n\n CHANGELOG')
    print(change_log)
    print("finished!")

    return change_log
    
#change_log = run_bot()

In [ ]:
def display_all_tables():
    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    for table in tables:
        print(f"Table: {table[0]}")
        cursor.execute(f"SELECT * FROM {table[0]}")
        rows = cursor.fetchall()
        for row in rows:
            print(row)
        print("-" * 40)
    conn.close()
display_all_tables()

In [ ]:
def split_change_log(text, max_length=9600):
    parts = []
    part_number = 1

    while len(text) > max_length:
        split_index = text.rfind('\n', 0, max_length)
        if split_index == -1:  # If no newline is found, force split
            split_index = max_length
        
        part_text = text[:split_index]
        formatted_text = (
            f"*These are the actions I performed since last post (part {part_number}).*\n\n"
            "---\n"
            f"{part_text}\n\n"
            "---\n"
            "^(These actions were performed automatically by a bot. If you think I made a mistake, respond to this comment. "
            "This will summon Aart, my creator.)"
        )
        parts.append(formatted_text)
        text = text[split_index+1:]  # Skip the newline character
        part_number += 1

    # Add the last remaining part
    formatted_text = (
        f"*These are the actions I performed since last post (part {part_number}).*\n\n"
        "---\n"
        f"{text}\n\n"
        "---\n"
        "^(These actions were performed automatically by a bot. If you think I made a mistake, respond to this comment. "
        "This will summon Aart, my creator.)"
    )
    parts.append(formatted_text)

    return parts

In [ ]:
def publish_post(change_log):
    explanation_text = '''You are looking at the first fully bot-run daily game: **DailyTrade**!\n\n
**How It Works**\n\n
The rules may seem complicated, but it’s actually pretty simple.\n\n
DailyTrade is a stock trading game—but instead of companies, you’re investing in **subreddits**! Stock values are based on how many posts appeared in that subreddit the previous day. You can join by simply announcing your first trade. You get a free starting budget of **1000 gems** to trade with.\n\n
**Example Trade**\n\n
- On **Day 1**, I buy **400 gems** worth of r/notinteresting stock.\n
- Between **Day 0 and Day 1**, there were **50 posts** on r/notinteresting.\n
- On **Day 3**, I decide to sell my stock.\n
- Between **Day 3 and Day 4**, there were **100 posts** on r/notinteresting.\n
- Since the number of posts **doubled**, my stock value doubles as well—I get **800 gems** back.\n
Of course, stock values can go down too—if the number of posts drops, you’ll lose gems when you sell.
\n\n---\n
**Bot Commands**\n\n
The bot reads and processes comments, so please follow these formatting rules carefully.\n\n
- **Always use square brackets** [ ] around commands so the bot knows you’re talking to it.\n
- You can include **regular text** in your comment too—the bot will only process text inside brackets.\n
- Commands can be **chained** (e.g., [buy 400 r/dailygames] blab la [sell 200 r/notinteresting]), and they will be executed in order.\n
**Available Commands**\n\n
- **Buy Stocks**: [buy AMOUNT r/SUBREDDIT]. Example: [buy 400 r/notinteresting] buys 400 gems worth of r/notinteresting stock. Your stocks are valued based on when *you* buy them (even if someone else buys later at a different price).\n
- **Sell Stocks**: [sell AMOUNT r/SUBREDDIT]. Example: [sell 400 r/notinteresting] sells 400 stocks of r/notinteresting at the current rate. You can also sell everything at once: [sell all r/notinteresting].\n
- **Take a Loan**: [loan AMOUNT]. Example: [loan 1000] takes a 1000-gem loan. Interest is **5% per day**, deducted automatically at the start of each day.\n
- **Repay a Loan**: [pay AMOUNT]. Example: [pay 500] pays back 500 gems toward your loan. You can also repay everything at once: [pay all].
- **Stop the game**: [exit] causes the bot to delete all of your information. You can always join again later.
\n\n---\n
**Game Rules & Notes**
- You **cannot** buy extra stocks from a subreddit if you already own some. You must sell all your stocks in that subreddit first.\n
- You can only buy stocks from certain subreddits, you can find them in one of the images. You can request additional subreddits by contacting me (or responding to this post).\n
- **Stock values update at 5 AM GMT** each day, based on the previous 24 hours. Posting time may vary slightly, but calculations are always consistent.\n
- **You can’t influence stock prices by posting** in a subreddit yourself (your own posts are ignored in the post count). Any attempts at ‘insider trading’—like using an alt account to inflate stock values—will be investigated by the Reddit IRS (a.k.a. me).\n
- You can only trade in **whole** number of gems and stocks.\n
- Stock value when **buying** is based on the *previous* day, stock **selling** is based on the *coming* day.\n
Let me know if you have any questions. Happy trading!\n\n
^(This post, and everything in it, was created automatically by a bot. If you think I made a mistake, respond to this post. This will summon Aart, my creator.)
'''
    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    cursor.execute("""
        SELECT COUNT(post_id)
        FROM posts
        LIMIT 1
    """)
    post_count = cursor.fetchone()[0] + 1
    conn.close()
    
    post_id, post_date = get_latest_post()
    submission = reddit.submission(id=post_id)

    flair_template_id = next(item['flair_template_id'] for item in submission.flair.choices() if item['flair_text'] == '[Serious]')

    subreddit = reddit.subreddit("dailygames")

    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    query = "SELECT username, amount FROM loans"
    loans_df = pd.read_sql_query(query, conn)
    conn.close()
    
    if len(loans_df) > 0:
        images = [{"image_path":"gems.png"},
        {"image_path":"stocks.png"},
        {"image_path":"loans.png"},
        {"image_path":"virtual worth.png"},
        {"image_path":"allowed subreddits.png"}]
    else:
        images = [{"image_path":"gems.png"},
        {"image_path":"stocks.png"},
        {"image_path":"virtual worth.png"},
        {"image_path":"allowed subreddits.png"}]

    # Submit a post
    post = subreddit.submit_gallery(images=images,
    title="DailyTrade day " + str(post_count),
    flair_id=flair_template_id)

    print(f"Post created: {post.url} - {post.id}")

    submission = reddit.submission(id=post.id)
    explanation = submission.reply(explanation_text)
    print(f"Explanation posted: {explanation.id}")

    conn = sqlite3.connect("reddit_game.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO posts (post_id, date) VALUES (?, ?)", (post.id, get_today()))
    conn.commit()
    cursor.execute("INSERT INTO comments (comment_id, date) VALUES (?, ?)", (explanation.id, get_today()))
    conn.commit()
    if len(change_log) > 9600:
        for log_part in split_change_log(change_log):
            log = submission.reply(log_part)
            print(f"Part of log posted: {log.id}")
            cursor.execute("INSERT INTO comments (comment_id, date) VALUES (?, ?)", (log.id, get_today()))
            conn.commit()
    else:
        change_log = (
            f"*These are the actions I performed since last post.*\n\n"
            "---\n"
            f"{change_log}\n\n"
            "---\n"
            "^(These actions were performed automatically by a bot. If you think I made a mistake, respond to this comment. "
            "This will summon Aart, my creator.)"
        )
        log = submission.reply(change_log)
        print(f"Log posted: {log.id}")
        cursor.execute("INSERT INTO comments (comment_id, date) VALUES (?, ?)", (log.id, get_today()))
        conn.commit()
    conn.close
    print("Finished!")

publish_post(change_log)